<a href="https://colab.research.google.com/github/schettinina/MONAN---Back-end/blob/main/Testando_novo_artigo_09_11_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install ollama pandas requests PyPDF2

In [34]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [35]:
import subprocess
import time
subprocess.Popen(['ollama', 'serve'])
time.sleep(5)  # Aguardar inicialização

In [36]:
!ollama pull deepseek-coder-v2

In [40]:
import pandas as pd
import subprocess
import os
import requests
from io import BytesIO
import PyPDF2
import time
import re

# Função para baixar e extrair texto de um PDF do GitHub q contem o artigo selecionado
def get_pdf_text_from_github(github_url):
    try:
        if 'github.com' in github_url and '/blob/' in github_url:
            raw_url = github_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
        else:
            raw_url = github_url

        print(f"📥 Baixando de: {raw_url}")
        response = requests.get(raw_url)
        response.raise_for_status()

        pdf_reader = PyPDF2.PdfReader(BytesIO(response.content))
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

        print(f"✅ PDF processado - {len(text)} caracteres extraídos")
        return text

    except Exception as e:
        print(f"❌ Erro ao processar PDF: {str(e)}")
        return ""

# Função para consultar o a llm do DeepSeek
def perguntar_ao_deepseek(pergunta):
    try:
        comando = f'ollama run deepseek-coder:6.7b "{pergunta}"'
        resultado = subprocess.run(comando, shell=True, capture_output=True, text=True, timeout=90)

        if resultado.returncode == 0:
            return resultado.stdout.strip()
        else:
            return "Erro na consulta"

    except subprocess.TimeoutExpired:
        return "Tempo limite excedido"
    except Exception as e:
        return f"Erro: {str(e)}"

# Perguntas do AMSTAR 2 com critérios específicos Aprimorada para melhotr resposta
amstar_questions = {
    1: {
        "pergunta": "A pergunta de pesquisa e critérios de inclusão incluem os componentes PICO?",
        "critérios": ["População", "Intervenção", "Comparação", "Desfecho", "PICO explicitado"]
    },
    2: {
        "pergunta": "Os métodos da revisão foram estabelecidos antes da condução e há justificativa para desvios?",
        "critérios": ["Protocolo pré-registrado", "Justificativa para desvios", "Métodos predefinidos"]
    },
    3: {
        "pergunta": "Os autores justificaram a seleção dos desenhos de estudo incluídos?",
        "critérios": ["Justificativa para desenhos", "Critérios de inclusão explicados"]
    },
    4: {
        "pergunta": "Foi utilizada uma estratégia abrangente de busca na literatura?",
        "critérios": ["Múltiplas bases", "Estratégia de busca detalhada", "Sem restrição de idioma", "Busca na literatura cinzenta"]
    },
    5: {
        "pergunta": "A seleção de estudos foi realizada em duplicata?",
        "critérios": ["Dois revisores independentes", "Processo de consenso", "Fluxo de seleção descrito"]
    },
    6: {
        "pergunta": "A extração de dados foi realizada em duplicata?",
        "critérios": ["Dois extratores", "Formulário padronizado", "Verificação cruzada"]
    },
    7: {
        "pergunta": "O artigo apresenta lista dos estudos excluídos com motivos?",
        "critérios": ["Lista de excluídos", "Motivos detalhados", "Fluxo PRISMA"]
    },
    8: {
        "pergunta": "Os estudos incluídos foram descritos de maneira completa?",
        "critérios": ["Características dos estudos", "Tabela descritiva", "Dados demográficos"]
    },
    9: {
        "pergunta": "Foi utilizada uma técnica adequada para avaliar o risco de viés nos estudos incluídos?",
        "critérios": [
            "Uso de ferramenta validada para avaliação de viés",
            "Dois avaliadores independentes para avaliação de viés",
            "Descrição clara dos critérios de avaliação de viés",
            "Apresentação dos resultados da avaliação de viés",
            "Menção específica à ferramenta utilizada (ex: QualSyst, Cochrane RoB)"
        ]
    },
    10: {
        "pergunta": "Os autores relataram fontes de financiamento dos estudos incluídos?",
        "critérios": ["Fontes de financiamento", "Conflitos de interesse dos estudos primários"]
    },
    11: {
        "pergunta": "Se foi realizada meta-análise, os métodos estatísticos utilizados foram apropriados?",
        "critérios": [
            "Uso de modelo estatístico adequado (efeitos fixos/aleatórios)",
            "Justificativa para escolha do modelo estatístico",
            "Métodos apropriados para combinação de dados",
            "Cálculo de medidas de efeito apropriadas",
            "Descrição clara dos métodos de análise estatística"
        ]
    },
    12: {
        "pergunta": "Se foi realizada meta-análise, os autores avaliaram o impacto do risco de viés nos resultados?",
        "critérios": [
            "Análise de sensibilidade considerando o risco de viés",
            "Comparação entre estudos de alto vs. baixo risco de viés",
            "Discussão sobre como o viés pode ter afetado os resultados",
            "Avaliação do impacto do viés na força das evidências",
            "Menção explícita à relação entre viés e resultados"
        ]
    },
    13: {
        "pergunta": "Os autores consideraram o risco de viés ao interpretar os resultados?",
        "critérios": ["Discussão do viés", "Limitações mencionadas", "Interpretação considerando viés"]
    },
    14: {
        "pergunta": "Houve explicação satisfatória para qualquer heterogeneidade observada?",
        "critérios": ["Análise de heterogeneidade", "I² calculado", "Exploração das causas"]
    },
    15: {
        "pergunta": "Se foi realizada meta-análise, os autores investigaram adequadamente o viés de publicação?",
        "critérios": [
            "Uso de métodos para detectar viés de publicação (ex: funnel plot)",
            "Aplicação de testes estatísticos para viés de publicação",
            "Busca abrangente incluindo literatura cinzenta",
            "Discussão sobre possível impacto do viés de publicação",
            "Estratégias para minimizar viés de publicação"
        ]
    },
    16: {
        "pergunta": "Os autores relataram adequadamente os conflitos de interesse e fontes de financiamento?",
        "critérios": [
            "Declaração explícita de conflitos de interesse dos autores",
            "Identificação das fontes de financiamento da revisão",
            "Menção ao papel dos financiadores no estudo",
            "Declaração sobre independência dos pesquisadores",
            "Transparência completa sobre relações financeiras"
        ]
    }
}

def criar_prompt_especifico(numero, pergunta_data, pdf_text):
    """Cria prompt específico para cada pergunta do AMSTAR 2"""

    pergunta = pergunta_data["pergunta"]
    criterios = pergunta_data["critérios"]

    # Trecho relevante do PDF baseado na pergunta
    contexto = pdf_text[:10000]  # Limitar contexto

    prompt = f"""
## INSTRUÇÕES ESPECÍFICAS:
Você é um avaliador especializado em revisões sistemáticas usando a ferramenta AMSTAR 2.
Analise CRITERIOSAMENTE o artigo e responda APENAS com SIM, PARCIALMENTE ou NÃO.

## PERGUNTA {numero}: {pergunta}

## CRITÉRIOS A SEREM AVALIADOS:
{chr(10).join(f"- {criterio}" for criterio in criterios)}

## TEXTO DO ARTIGO PARA ANÁLISE:
{contexto}

## REGRAS DE AVALIAÇÃO:
- **SIM**: Todos os critérios principais estão claramente atendidos e descritos
- **PARCIALMENTE**: Alguns critérios estão atendidos, mas há lacunas importantes
- **NÃO**: Critérios não atendidos ou informações insuficientes

## EXEMPLOS:
- Se o artigo descrever "dois revisores independentes" para seleção → SIM
- Se mencionar "seleção por um revisor" → PARCIALMENTE
- Se não mencionar processo de seleção → NÃO

## RESPOSTA FINAL (APENAS UMA PALAVRA):
"""
    return prompt

def avaliar_amstar2(pdf_text):
    print("\n🔍 Avaliando o artigo com AMSTAR 2...")
    print("⏳ Isso pode levar alguns minutos...")

    resultados = {}
    justificativas = {}

    for numero, pergunta_data in amstar_questions.items():
        print(f"📋 Processando pergunta {numero}/16...")

        prompt = criar_prompt_especifico(numero, pergunta_data, pdf_text)
        resposta = perguntar_ao_deepseek(prompt).upper().strip()

        print(f"   Resposta bruta: {resposta}")

        # Processamento mais robusto da resposta
        resposta_limpa = re.sub(r'[^A-ZÀ-Ú\s]', '', resposta).strip()

        if re.search(r'\b(SIM|YES)\b', resposta_limpa, re.IGNORECASE):
            resposta_final = "SIM"
        elif re.search(r'\b(PARCIALMENTE|PARCIAL|PARTIAL)\b', resposta_limpa, re.IGNORECASE):
            resposta_final = "PARCIALMENTE"
        elif re.search(r'\b(NÃO|NAO|NO)\b', resposta_limpa, re.IGNORECASE):
            resposta_final = "NÃO"
        else:
            # Análise de contexto para respostas ambíguas
            palavras_positivas = ['atendido', 'sim', 'yes', 'presente', 'descrito', 'claramente']
            palavras_parciais = ['parcial', 'alguns', 'mas', 'porém', 'entretanto']

            if any(palavra in resposta.lower() for palavra in palavras_positivas):
                resposta_final = "SIM"
            elif any(palavra in resposta.lower() for palavra in palavras_parciais):
                resposta_final = "PARCIALMENTE"
            else:
                resposta_final = "NÃO"

        resultados[numero] = resposta_final
        justificativas[numero] = resposta

        print(f"   ✅ {numero:2d}. {resposta_final}")
        time.sleep(3)  # Pausa maior entre perguntas

    return resultados, justificativas

def calcular_score(resultados):
    score = 0
    for r in resultados.values():
        if r == "SIM":
            score += 1
        elif r == "PARCIALMENTE":
            score += 0.5
    return score

def analisar_padrao_respostas(resultados):
    """Analisa padrões nas respostas para detectar viés"""
    total_sim = sum(1 for r in resultados.values() if r == "SIM")
    total_nao = sum(1 for r in resultados.values() if r == "NÃO")
    total_parcial = sum(1 for r in resultados.values() if r == "PARCIALMENTE")

    print(f"\n📈 ESTATÍSTICAS DAS RESPOSTAS:")
    print(f"   SIM: {total_sim}/16 ({total_sim/16*100:.1f}%)")
    print(f"   PARCIALMENTE: {total_parcial}/16 ({total_parcial/16*100:.1f}%)")
    print(f"   NÃO: {total_nao}/16 ({total_nao/16*100:.1f}%)")

    # Alertas de qualidade
    if total_nao >= 8:
        print("   ⚠️  Muitos critérios não atendidos - revisão pode ter limitações significativas")
    if total_sim >= 12:
        print("   ✅ Boa aderência aos critérios AMSTAR 2")

def main():
    print("""
    ====================================
    🎯 AVALIADOR AVANÇADO - AMSTAR 2
    ====================================
    """)

    # Solicitar URL do usuário
    pdf_url = input("🔗 Cole a URL do PDF no GitHub: ").strip()

    # Baixar e extrair texto do PDF
    print(f"\n📥 Acessando PDF...")
    pdf_text = get_pdf_text_from_github(pdf_url)

    if not pdf_text:
        print("❌ Falha ao extrair texto do PDF")
        return

    print(f"✅ Texto extraído: {len(pdf_text)} caracteres")

    # Avaliação AMSTAR 2
    resultados, justificativas = avaliar_amstar2(pdf_text)

    if resultados:
        print("\n" + "="*70)
        print("📊 RESULTADOS FINAIS AMSTAR 2")
        print("="*70)

        for num, pergunta_data in amstar_questions.items():
            resp = resultados[num]
            pergunta = pergunta_data["pergunta"]
            print(f"{num:2d}. {resp:15} - {pergunta}")

        # Cálculo e análise
        score = calcular_score(resultados)
        percentual = (score / 16) * 100

        print(f"\n🔢 PONTUAÇÃO: {score:.1f}/16 ({percentual:.1f}%)")

        # Classificação de qualidade
        if score >= 14:
            qualidade = "ALTA QUALIDADE"
            cor = "🟢"
        elif score >= 11:
            qualidade = "QUALIDADE MODERADA"
            cor = "🟡"
        elif score >= 8:
            qualidade = "QUALIDADE BAIXA"
            cor = "🟠"
        else:
            qualidade = "QUALIDADE MUITO BAIXA"
            cor = "🔴"

        print(f"🏆 CLASSIFICAÇÃO: {cor} {qualidade}")

        # Análise de padrões
        analisar_padrao_respostas(resultados)

        # Salvar resultados detalhados
        df = pd.DataFrame({
            'Pergunta_Numero': list(amstar_questions.keys()),
            'Pergunta': [data["pergunta"] for data in amstar_questions.values()],
            'Resposta': [resultados[n] for n in amstar_questions.keys()],
            'Pontuacao': [1 if resultados[n] == "SIM" else 0.5 if resultados[n] == "PARCIALMENTE" else 0 for n in amstar_questions.keys()],
            'Justificativa_LLM': [justificativas[n] for n in amstar_questions.keys()],
            'Critérios_Avaliados': [", ".join(data["critérios"]) for data in amstar_questions.values()]
        })

        nome_arquivo = "avaliacao_amstar2_detalhada.csv"
        df.to_csv(nome_arquivo, index=False, encoding='utf-8')
        print(f"\n💾 CSV detalhado salvo: {nome_arquivo}")

        # Salvar relatório executivo
        with open("relatorio_executivo.txt", "w", encoding="utf-8") as f:
            f.write("RELATÓRIO EXECUTIVO - AVALIAÇÃO AMSTAR 2\n")
            f.write("="*60 + "\n\n")
            f.write(f"URL do artigo: {pdf_url}\n")
            f.write(f"Pontuação total: {score:.1f}/16 ({percentual:.1f}%)\n")
            f.write(f"Classificação: {qualidade}\n\n")

            f.write("PONTOS FORTES:\n")
            for num, resp in resultados.items():
                if resp == "SIM":
                    f.write(f"✓ {amstar_questions[num]['pergunta']}\n")

            f.write("\nPONTOS FRACOS:\n")
            for num, resp in resultados.items():
                if resp == "NÃO":
                    f.write(f"✗ {amstar_questions[num]['pergunta']}\n")

            f.write("\nÁREAS DE MELHORIA (PARCIALMENTE):\n")
            for num, resp in resultados.items():
                if resp == "PARCIALMENTE":
                    f.write(f"~ {amstar_questions[num]['pergunta']}\n")

        print("📄 Relatório executivo salvo: relatorio_executivo.txt")

        print("\n✅ Análise concluída com sucesso!")

# Executar
if __name__ == "__main__":
    main()


    🎯 AVALIADOR AVANÇADO - AMSTAR 2
    
🔗 Cole a URL do PDF no GitHub: https://github.com/schettinina/MONAN_artigosparallm/blob/main/Looking%20beyond%20supply_%20a%20systematic%20literature%20review%20of%20demand-side%20barriers%20to%20health%20service%20utilization%20in%20the%20mountains%20of%20Nepal%20-%20PubMed.pdf

📥 Acessando PDF...
📥 Baixando de: https://raw.githubusercontent.com/schettinina/MONAN_artigosparallm/main/Looking%20beyond%20supply_%20a%20systematic%20literature%20review%20of%20demand-side%20barriers%20to%20health%20service%20utilization%20in%20the%20mountains%20of%20Nepal%20-%20PubMed.pdf
✅ PDF processado - 2029 caracteres extraídos
✅ Texto extraído: 2029 caracteres

🔍 Avaliando o artigo com AMSTAR 2...
⏳ Isso pode levar alguns minutos...
📋 Processando pergunta 1/16...
   Resposta bruta: SIM
   ✅  1. SIM
📋 Processando pergunta 2/16...
   Resposta bruta: NÃO. O ARTIGO NÃO ESPECIFICA OS MÉTODOS DA REVISÃO ANTES DA CONDUÇÃO E NÃO APRESENTA JUSTIFICATIVA SUFICIENTE PARA